In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [20]:
# Class to perform Weighted Least Squares (WLS) estimation
class WLS_estimator:
    def __init__(self, file_name, i, j):
        # Constructor code here
        self.file_name = file_name
        self.i = i
        self.j = j

    def get_train_values(self):
        # Outputs the values for the selected i and j
        self.raw_data = pd.read_csv(self.file_name, header=None, names=[
                                    'velocity_l', 'velocity_r', 'beta_l', 'beta_r', 'alpha', 'i', 'j'])
        self.data = self.raw_data[(self.raw_data['i'] == self.i) & (
            self.raw_data['j'] == self.j)]
        self.raw_data = pd.read_csv(self.file_name)
        sensor_readings = []
        target_values = []
        for index, row in self.data.iterrows():
            # Take velocity_r and velocity_l
            sensor_readings.append(row[['velocity_r', 'velocity_l']].values)
            target_values.append(row[['beta_r', 'beta_l', 'alpha']].values)  #
        self.sensor_readings_array = np.array(sensor_readings)
        self.target_values_array = np.array(target_values)
        return self.sensor_readings_array, self.target_values_array

    def wls_operation(self, X, R, y):
        """Perform WLS to predict the parameter."""
        # Compute the weights
        # Weight matrix is the inverse of the covariance matrix
        W = np.linalg.inv(R)
        X_T_W_X = X.T @ W @ X
        X_T_W_y = X.T @ W @ y
        # Compute the WLS estimate
        beta = np.linalg.inv(X_T_W_X) @ X_T_W_y
        return beta

    def apply_wls(self):

        np.random.seed(0)
        X, y = self.get_train_values()
        if X.shape[0] == 0:
            print("No data found for the selected i and j")
            return        # Covariance matrix of the measurement noise
        # Random noise for each measurement
        sigma_eps = np.random.rand(X.shape[0])*1e-1
        # Covariance matrix for a single measurement
        R_eps = np.diag(sigma_eps ** 2)
        R = R_eps
        # Number of measurements
        n_meas = X.shape[0]
        beta = self.wls_operation(X, R, y)
        self.beta = beta
        print("predicted beta: ", beta)
        return beta

    def wls_predict(self, X):
        if X.shape[0] == 0:
            print("No data found for the selected i and j")
            return
        predictions = X @ self.beta
        # print("predictions: ", predictions)
        return predictions

    def compute_error(self, predictions, y):
        errors = []

        if len(predictions.shape) > 1:
            for i in range(len(self.sensor_readings_array)):
                errors.append(abs(predictions[i])-abs(y[i]))
            errors = np.array(errors)

            for i in range(len(predictions)):
                print(
                    f"Predicted: {predictions[i]}, Actual: {y[i]}, Error: {errors[i]}")
                # print(f"Percentage error: {abs(errors[i])/abs(y[i])*100}%")
        else:
            errors = predictions - y  # This would give the error as mentioned
            print(f"Predicted: {predictions}, Actual: {y}, Error: {errors}")
            # Compute the absolute error
            # absolute_error = np.abs(errors)
            # Compute the percentage error
            # percentage_error = (absolute_error / np.abs(y)) * 100

            # print(f"Percentage error: {percentage_error}%")

In [21]:
class MapWLS:
    def __init__(self, map_height, map_width):
        self.map_height = map_height
        self.map_width = map_width
        self.wls_estimators = [[WLS_estimator('../data/robot_data.csv', i, j) for j in range(map_height)] for i in range(map_width)]

    def check_map(self):
        for i in range(self.map_width):
            for j in range(self.map_height):
                print(f"i: {i}, j: {j}")
                wls = self.wls_estimators[i][j]
                wls.apply_wls()
                if wls.get_train_values()[0].shape[0] == 0:
                    continue
                
                predictions = wls.wls_predict(wls.get_train_values()[0][-1])
                wls.compute_error(predictions, wls.get_train_values()[1][-1])
                print("\n")
                
    def get_estimate(self, i, j,value):
        return self.wls_estimators[i][j].wls_predict(value)
    
    
    def plot_predictios(self,i,j):
        print("Plotting the predictions")
        wls = self.wls_estimators[i][j]
        X, y = wls.get_train_values()
        predictions = wls.wls_predict(X)
        target_values_array_test = y
        n_columns = 3  # since the array has 3 columns
        names=['beta_r','beta_l','alpha']
        plt.figure(figsize=(12, 8))
        for i in range(n_columns):
            plt.subplot(3, 1, i+1)  # Create a subplot for each column (3 rows, 1 column)
            plt.plot(predictions[:, i], label=f'{names[i]} Predicted', color='blue', linestyle='--')
            plt.plot(target_values_array_test[:, i], label=f'{names[i]} Target', color='red',linestyle='dotted')
            plt.xlabel('Sample')
            plt.ylabel(f'{names[i]} value')
            plt.title(f'{names[i]} Values')
            plt.legend()
        plt.tight_layout()  # Adjust layout to prevent overlapping
        plt.show() 

In [22]:
mapwls = MapWLS(10,10)

In [23]:
mapwls.check_map()

i: 0, j: 0
No data found for the selected i and j
i: 0, j: 1
No data found for the selected i and j
i: 0, j: 2
No data found for the selected i and j
i: 0, j: 3
No data found for the selected i and j
i: 0, j: 4
No data found for the selected i and j
i: 0, j: 5
No data found for the selected i and j
i: 0, j: 6
No data found for the selected i and j
i: 0, j: 7
No data found for the selected i and j
i: 0, j: 8
No data found for the selected i and j
i: 0, j: 9
No data found for the selected i and j
i: 1, j: 0
No data found for the selected i and j
i: 1, j: 1
No data found for the selected i and j
i: 1, j: 2
No data found for the selected i and j
i: 1, j: 3
predicted beta:  [[ 0.00772037 -0.00777088 -0.00093212]
 [-0.00934482  0.00921302  0.00160341]]
Predicted: [ 0.00746034 -0.00793974  0.00015968], Actual: [ 0.0072647  -0.00785183  0.        ], Error: [ 1.95643135e-04 -8.79146932e-05  1.59681627e-04]


i: 1, j: 4
predicted beta:  [[ 0.00523882 -0.0053623  -0.00688358]
 [-0.00510415  0.005